In [1]:
import requests
import time
import os
from openai import AzureOpenAI

def api_call_gpt_with_retry(message_content:str, max_retries=3, delay=2):

    client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-05-15"
    )

    system_message = f'''
    You are a excellent translator from Norwegian to British English (United Kingdom). 
    You are translating Norwegian consultant reports into english, so your language should be direct and professional.
    DO NOT INCLUDE additional text, only translate the words provided in the prompt.
    '''

    prompt = f'''Translate the following text into British English: {message_content}\n'''
    messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ]
    

    for attempt in range(max_retries): 
        response = client.chat.completions.create(
            model= os.getenv("MODEL_NAME"), # model = "deployment_name".
            messages=messages)

        try:
        
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error {e}")
            print(response.text)
            if attempt < max_retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Exiting.")

In [2]:
# Import necessary modules
import os
from dotenv import load_dotenv
from pathlib import Path

# Get the folder path of the Jupyter Notebook file
notebook_path = os.path.abspath("__file__")
parent_folder = os.path.dirname(notebook_path)
project_folder = os.path.join(parent_folder, "Rapporter - Original")
ferdig_dokumenter_path = os.path.join(parent_folder,"Maskinoversatte sammendrag")

# Set the name of the Word document to translate
dokumentnavn = r"Sjeldne diagnoser og prioritering notat utkast Menon 9 februar 2023"

# Create the full path to the Word document
doc_to_translate = os.path.join(project_folder, f"{dokumentnavn}.docx")

# Set the path to the folder where translated documents will be saved


# Path to your .env file (replace with the actual path)
env_path = os.path.join(parent_folder, '.env')

# Load the .env file to get the OpenAI API key
load_dotenv(dotenv_path=env_path)
api_key = os.getenv('OPENAI_API_KEY')



In [3]:
print(api_call_gpt_with_retry("Hei, hvordan går det?"))

Hello, how are you doing?


In [4]:
from docx import Document

def apply_style(doc, text, style_name):
    # Create a new paragraph
    paragraph = doc.add_paragraph()

    # Add the text to the paragraph
    paragraph.add_run(text)

    # Apply the style if it exists in the document's style library
    if style_name in doc.styles:
        paragraph.style = doc.styles[style_name]
    else:
        paragraph.style = 'Normal'

def read_and_manipulate_docx(file_path):
    try:
        doc = Document(file_path)
    except Exception as e:
        print(f"ERROR: Problems encountered when trying to open the document ({e}). Is it open on your computer?")
        return

    collected_content = []
    placeholder_img_path = "placeholder.png"

    # Extract text and style from each paragraph in the original document
    for paragraph in doc.paragraphs:
        collected_content.append((paragraph.text, paragraph.style.name))

    # Create a new document
    new_doc = Document()
    for text, style in collected_content:
        if text.strip():
            print(f"Translating: {text}")
            translated_text = api_call_gpt_with_retry(text)

            # Handle captions and images
            if style == 'Caption':
                print(f"Adding caption: {text}")
                apply_style(new_doc, translated_text, style)

                # Add placeholder image
                picture_paragraph = new_doc.add_paragraph()
                picture_run = picture_paragraph.add_run()
                picture_run.add_picture(placeholder_img_path)
            else:
                # Apply styles for text and headings
                apply_style(new_doc, translated_text, style)
        else:
            # Add a line break
            new_doc.add_paragraph()

    return new_doc

# Usage example
# read_and_manipulate_docx('path_to_your_document.docx')


In [5]:
# Get the list of files in the project folder
file_list = os.listdir(project_folder)
translated_list = os.listdir(ferdig_dokumenter_path)

# Print the file names
for file_name in file_list:
    trans_name = f'{file_name.rsplit(sep=".", maxsplit=1)[0]}_translated_summary.docx'
    
    print(trans_name)
    if trans_name in translated_list:
        print(f"Skipping: {file_name}")
    elif f"{trans_name.split('_', maxsplit=1)[0]}_translated_summary.docx" in translated_list:
        print(f"Skipping: {file_name}")
    else:
        print(f"Translating: {file_name}")
        doc_to_translate = os.path.join(project_folder, file_name)
        oversatt_dok = read_and_manipulate_docx(doc_to_translate)
        
        try:
            oversatt_dok.save(f"{ferdig_dokumenter_path}\\{trans_name}")
        except FileNotFoundError:
            print(f"Lagrer dokumentet etter prosjektnummer")
            oversatt_dok.save(f"{ferdig_dokumenter_path}\\{trans_name.split('_', maxsplit=1)[0]}_translated_summary.docx")
            continue

        del oversatt_dok

Notat Automatiseringspotensial i Norge_translated_summary.docx
Translating: Notat Automatiseringspotensial i Norge.docx
Translating: Forord 
Translating: Det er disse månedene mye snakk om KI og hvordan det vil påvirke samfunnet. I dette notatet forsøker vi å kvantifisere hvilken innvirkning dagens tilgjengelige KI-verktøy vil kunne ha på det norske samfunnet hvis de tas i bruk. Konkret ser vi på potensialet for å effektivisere tid brukt på å løse dagens arbeidsoppgaver. Teknologi generelt, og kunstig intelligens spesielt, har potensial til å omforme hvordan vi jobber og lever, og vi kan ikke ennå forutsi hvordan en slik effektiviseringsgevinst tas ut. John Maynard Keynes spådde for snart 100 år siden at teknologisk vekst ville tas ut i økt fritid, men det har vi ennå ikke sett. I notatet forsøker vi derfor ikke å si noe om hvordan, men hvor mye.
Translating: Mange har forsøkt før oss, men det unike med vår fremgangsmåte er at vi kobler etablert internasjonal forskningsmetodikk med nor

# Appendix

In [3]:
import os
from docx import Document

original_folder = "Rapporter - Original"
translated_folder = "Maskinoversatte sammendrag"
output_folder = "Ferdig oversatte sammendrag"

# Get the list of files in the original folder
original_files = os.listdir(original_folder)

# Loop over the original files
for file_name in original_files:
    # Get the full path of the original file
    original_file_path = os.path.join(original_folder, file_name)
    
    # Load the original document
    original_doc = Document(original_file_path)
    
    # Get the style of the first page
    first_page_style = original_doc.sections[0].header.paragraphs[0].style

    filesplit = file_name.rsplit(sep=".", maxsplit=1)[0]
    
    # Get the corresponding translated file
    translated_file_path = os.path.join(translated_folder, f"{filesplit}_translated_summary.docx")
    
    # Load the translated document
    translated_doc = Document(translated_file_path)
    
    # Loop over the pages of the translated document
    for page in translated_doc.sections:
        # Add the page to the original document
        original_doc.add_section(page)
    
    # Set the style of the first page in the original document
    original_doc.sections[0].header.paragraphs[0].style = first_page_style
    
    # Save the modified document to the output folder with the same name
    output_file_path = os.path.join(output_folder, file_name)
    original_doc.save(output_file_path)


ValueError: value '<docx.section.Section object at 0x0000025FD6ABBB50>' not in enumeration WD_SECTION_START

In [3]:
from docx import Document

def read_and_manipulate_docx(file_path):
    try:
        doc = Document(file_path)
    except Exception as e:
        print(f"ERROR ERROR\n\nStøtte på problemer når vi forsøkte å åpne prosjektet, ERROR({e}).\nHar du det åpnet på din maskin?")
        return
    collect_text = False
    collected_content = []

    for paragraph in doc.paragraphs:
        if paragraph.style.name == 'Heading 1' and paragraph.text == "Sammendrag":
            collect_text = True
            continue
        if collect_text:
            if paragraph.style.name == 'Heading 1':
                break  # Stop if another Heading 1 is found
            # Collect the text and style for both headings and normal text
            collected_content.append((paragraph.text, paragraph.style.name))

    # Perform your operations on collected_content here
    # ...

    # Write back to document (either modify existing or create new)
    new_doc = Document()
    new_doc.add_paragraph('Summary', style='Heading 1')
    for text, style in collected_content:
        if text != "" and text != " " and text != "\n":
            print(f"Translating: {text}")
            translated_text = api_call_gpt_with_retry(text)

        
            if style.startswith('Heading'):
                # Make the string translated_text to only capitalize the first letter and lower case the rest
                translated_text = translated_text[0].upper() + translated_text[1:].lower()
            new_paragraph = new_doc.add_paragraph(translated_text)
            new_paragraph.style = new_doc.styles[style] if style in new_doc.styles else 'Normal'
        else:
            print('line_break')
            new_paragraph = new_doc.add_paragraph(text)
            new_paragraph.style = new_doc.styles[style] if style in new_doc.styles else 'Normal'

    return new_doc


In [10]:
from docx import Document



def print_doc_styles(file_path):
    doc = Document(file_path)
    for paragraph in doc.paragraphs:
        print(f"Text: {paragraph.text} \nStyle: {paragraph.style.name}\n")


# Get the list of files in the project folder
file_list = os.listdir(project_folder)
translated_list = os.listdir(ferdig_dokumenter_path)

# Print the file names
for file_name in file_list:
    trans_name = f'{file_name.rsplit(sep=".", maxsplit=1)[0]}_translated_summary.docx'
    
    print(trans_name)
    if trans_name in translated_list:
        print(f"Skipping: {file_name}")
    elif f"{trans_name.split('_', maxsplit=1)[0]}_translated_summary.docx" in translated_list:
        print(f"Skipping: {file_name}")
    else:
        print(f"Translating: {file_name}")
        doc_to_translate = os.path.join(project_folder, file_name)
        print_doc_styles(doc_to_translate)
        
      


Klimaregnskap og tiltaksanalyser Nordland 2030_translated_summary.docx
Skipping: Klimaregnskap og tiltaksanalyser Nordland 2030.docx
Kompetansebehov i hydrogennæringen 180823_translated_summary.docx
Translating: Kompetansebehov i hydrogennæringen 180823.docx
Text:  
Style: Normal

Text:  
Style: TOC Heading

Text:  
Style: TOC Heading

Text:  
Style: TOC Heading

Text: Forord  
Style: TOC Heading

Text: På oppdrag for Agder H2 Nettverk, Norsk Hydrogenforum, Arena H2Cluster, Universitet i Agder, Fagskolen i Agder, H2House og NORCE (H2komp prosjekt-team) har Menon Economics gjennomført en kartlegging av norsk hydrogennæringens kompetansebehov og utdanningstilbudet som finnes. Målet med oppdraget var å undersøke hvorvidt det foreligger et gap mellom hydrogennæringens nåværende og fremtidige kompetansebehov og de utdanningsmulighetene som finnes. 
Style: Normal

Text: Studien har vært ledet av Maren N. Basso med Henrik Foseid og Ada Lunde som prosjektmedarbeidere. Jonas Erraia har vært ans

In [9]:
from docx import Document

def read_and_manipulate_docx(file_path):
    try:
        doc = Document(file_path)
    except Exception as e:
        print(f"ERROR ERROR\n\nStøtte på problemer når vi forsøkte å åpne prosjektet, ERROR({e}).\nHar du det åpnet på din maskin?")
        return
    collect_text = False
    antall_figurer = 0
    collected_content = []
    placeholder_img_path = "placeholder.png"

    for paragraph in doc.paragraphs:
        # Collect the text and style for all paragraphs
        collected_content.append((paragraph.text, paragraph.style.name))

    # Perform your operations on collected_content here
    # ...

    # Write back to document (either modify existing or create new)
    new_doc = Document()
    for text, style in collected_content:
        if text.strip():  # This checks for non-empty strings more efficiently
            print(f"Translating: {text}")
            translated_text = api_call_gpt_with_retry(text)

            if style == 'Caption':
                print(f"Adding caption: {text}")
                # Create a new paragraph for the caption
                caption_paragraph = new_doc.add_paragraph()
                caption_run = caption_paragraph.add_run(translated_text)
                caption_paragraph.style = new_doc.styles[style] if style in new_doc.styles else 'Normal'

                # Create a new paragraph for the picture
                picture_paragraph = new_doc.add_paragraph()
                picture_run = picture_paragraph.add_run()
                picture_run.add_picture(placeholder_img_path)

            else:
                new_paragraph = new_doc.add_paragraph()
                new_paragraph.add_run(translated_text)

                if style.startswith('Heading'):
                    # Capitalize only the first letter
                    translated_text = translated_text[0].upper() + translated_text[1:].lower()
                    new_paragraph.style = new_doc.styles[style] if style in new_doc.styles else 'Normal'

                else:
                    new_paragraph.style = new_doc.styles[style] if style in new_doc.styles else 'Normal'

        else:
            print('line_break')
            new_paragraph = new_doc.add_paragraph()
            new_paragraph.style = new_doc.styles[style] 


    return new_doc